In [2]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Annotated, Sequence, Any, List, Dict, Optional
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_core.messages import ToolMessage

from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

# State
## 4 Question while detecting if it should be at state or not:
**other node needs this information?**


**Is This Information "Global" Information That Must Be Protected Throughout Its Entire Flow?**


**Will This Information Be Used to Decide Where the Stream Will Go?**

**Bu Bilgi, Birikmesi (Accumulate) Gereken Bir Bilgi mi?**

In [3]:
class HomeworkState(TypedDict):
    does_need_to_rewrite : bool
    researcher_result : List[str]
    writer_result : str
    mistakes : str
    messages : Annotated[Sequence[BaseMessage],add_messages]
    document_path: str

# Model

In [4]:
llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')

E0000 00:00:1761829246.094656     989 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


# Tools

In [5]:
from langchain_tavily import TavilySearch
search_runnable = TavilySearch(max_results = 5)
tools = [search_runnable]

In [6]:
llm_with_tools = llm.bind_tools(tools)

# Nodes

In [7]:
def Researcher_agent(state: HomeworkState) -> HomeworkState:
    """This search agent node recieves the topic and decides to search it on web."""

    print("--- RESEARCHER_agent WORKING ---")
    
    message = state['messages']

    response = llm_with_tools.invoke(message)
    #this will contain AIMessage about tool_call not answer of the llm query.

    #after tool run, the response will turn back to reasearcher node by "Conditional edge (we will set it later.)"
    # messages will be like this:
    #HumanMessage (.........)
    #AIMessage (..........)
    #ToolMessage (...........)
    #the second time llm ask itself:Are these search results sufficient? Or do I need more information?
    #Senerio A: This results enough.
    #Senerio B: These are not enough I want to use the tool again. AIMessage(tool_calls=[...New_query...])
    print("--- REASEARCHER FINISHED DRAFT ---")
    return {'messages':[response]}

In [8]:
def compile_research_node(state: HomeworkState) -> HomeworkState:
    """Runs after the search cycle completes.
    Finds all ToolMessages in the 'messages' list, collects their contents, and writes them to
    the 'researcher_result' key. """

    print("--- compile_research_node WORKING ---")

    
    compaile_results = []
    messages = state['messages']
    for m in messages:
        if isinstance(m, ToolMessage):
            # The content returned by TavilySearch may already be a list,
            # so it might be safer to use 'extend'.
            # OR 'append' if you assume the content is a string.
            if isinstance(m.content,str):
                compaile_results.append(m.content)

            elif isinstance(m.content, list):
                compaile_results.extend(m.content)
    print("--- COMPAILE FINISHED DRAFT ---")

    return {'researcher_result': compaile_results}

In [9]:
from langchain.schema.messages import SystemMessage, HumanMessage
def writer_agent(state: HomeworkState) -> HomeworkState: #google use docstring like this:
    """
Runs the Writer Agent to synthesize an academic draft in Markdown.

This node retrieves sources from the 'researcher_result' key in the state, 
prompts the LLM to write a formal draft based on those sources, and
formats the output as Markdown.

Args:
    state (Homework_state): The current state of the graph. Must 
                            contain 'researcher_result'.

Returns:
    dict: A dictionary with the key 'writer_result' containing the
          newly generated Markdown draft.
"""
    print("--- writer_agent WORKING ---")
    
    research_results = state['researcher_result'] # if you give this to llm directly it may not understand so we will give it more readable shape.
    sources_text = "\n\n---\n\n".join(research_results)
    
    does_need_to_rewrite = state.get('does_need_to_rewrite')
    
    if not does_need_to_rewrite or does_need_to_rewrite == None:        
        system_prompt = """
        You are an expert academic writer. 
        Your task is to synthesize information from various sources into a coherent, 
        well-structured academic paper.
        You must only use the information provided in the sources.
        You must format your entire output in Markdown.
        """
        
        user_prompt = f"""
        Here are the research sources:
        
        <SOURCES>
        {sources_text}
        </SOURCES>
        
        Please write an academic draft in Markdown based *only* on these sources.
        """
    
        #in here we don't use llm_with_tools because only reaseacher can use it.
         # If you use a chat model then you have to use : SystemMessage or HumanMessage. On Reasearch part we will give the input with using HumanMessages.
        #SystemMessage: Tells LLM who you are.
        #HumanMessage : Tells the request to LLM.

    else:
        writer_result = state['writer_result']
        mistakes = state['mistakes']
        
        system_prompt = """
        You are an expert academic editor. 
        Your task is to rewrite an academic paper to fix specific mistakes.
        You must use the <SOURCES> as the single source of truth.
        The new paper MUST be in Markdown.
        
        """
        user_prompt = f"""
        Here are the original research sources:
        
        <SOURCES>
        {sources_text}
        </SOURCES>
        
        Here is the flawed academic paper:
        <PAPER>
        {writer_result}
        </PAPER>

        Here are the mistakes you MUST fix:
        <MISTAKES>
        {mistakes}
        </MISTAKES>
        
        Please rewrite the entire paper in Markdown, ensuring all mistakes are 
        corrected and the content strictly follows the sources.
        """

    messages_for_llm = [
        SystemMessage(content = system_prompt),
        HumanMessage(content = user_prompt)
    ]
    
    response = llm.invoke(messages_for_llm)
        
    markdown_draft = response.content
        
    print("--- WRITER FINISHED DRAFT ---")
        
    return {"writer_result" : markdown_draft}

In [10]:
def controller_agent(state: HomeworkState) ->HomeworkState:
    """This agent controls the rewrite text and source text to detect if there is any hallucination on rewrited text writer_result
    
    Args:
    state (Homework_state): The current state of the graph. Must 
                            contain 'researcher_result' and 'writer_result '.

    
    Returns:
    bool: A boolean with the key 'does_need_to_rewrite' it will say if there is hallucination or not.                    
    """
    print('---CONTROLLER IS RUNNING---')
    researcher_result = state['researcher_result']
    writer_result = state['writer_result']
    
    research_results = state['researcher_result'] # if you give this to llm directly it may not understand so we will give it more readable shape.
    sources_text = "\n\n---\n\n".join(research_results)
    
    system_prompt = """You are an expert fact-checker and editor. Your task is to compare an academic paper
    against its original sources. You must identify *any* statements in the paper that
    are NOT supported by the sources (hallucinations) or contradict the sources.

    If the paper is perfect and has NO hallucinations, you must respond with 
    the single word: NONE
    
    If you find any hallucinations or unsupported claims, you MUST return a 
    list of the specific mistakes."""
    

    user_prompt = f"""
        Here are the research sources:
        
        <SOURCES>
        {sources_text}
        </SOURCES>
        Here are the rewrited academic paper:
        Here is the academic paper to check:
        <PAPER>
        {writer_result}
        </PAPER>
        
        Remember: Respond with ONLY the word "NONE" if there are no errors. Otherwise, list the errors.
        """
    messages_for_llm = [
            SystemMessage(content = system_prompt),
            HumanMessage(content = user_prompt)
        ]
    
    response = llm.invoke(messages_for_llm)
    response_content = response.content.strip()
    
    if response_content == "NONE":
        print('---THERE IS NO ERROR.')
        print('---CONTROLLER IS FINISHED---')
        return {'does_need_to_rewrite':False, 'mistakes': ""}
        
    else:
        print('---CONTROLLER FIND SOME ERROR:---')
        print('---CONTROLLER IS FINISHED---')
        
        return {'does_need_to_rewrite':True, 'mistakes': response_content}

In [11]:
import pypandoc
def formatter(state: HomeworkState) -> HomeworkState:
    """This is a formatter node. It creates a Word file from text written in Markdown format."""
    
    print('---FORMATTER NODE IS RUNNING ---')

    outputfile = 'student_number_homeworkname.docx'
    
    writer_result = state['writer_result']
    
    try:
        pypandoc.convert_text(
            writer_result,
            'docx',
            format = 'md', #markdown
            outputfile= outputfile
        )
        print(f"The {outputfile} saved succesfully!.")
        return {"document_path": outputfile}
        
    except Exception as e:
        print("---THERE IS SOMETHING WRONG THE WORD FILE COUND'T CREATE!---")
        print(e)
        return {"document_path": None}

# Edges

In [12]:
def should_contunie(state: HomeworkState) -> str:
    """The researcher decides the flow after the agent."""
    last_message = state['messages'][-1]
    
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return 'tool_call'

    else:
        return 'compile'

In [13]:
def decide_to_rewrite(state: HomeworkState) -> str:
    """Reads the Auditor's decision and directs the flow."""

    
    if state.get('does_need_to_rewrite') == True:
        return 'rewrite'
    else:
        return 'format'

In [14]:
workflow = StateGraph(HomeworkState)

In [15]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

workflow.add_node('researcher', Researcher_agent)
workflow.add_node('compile_research',compile_research_node)
workflow.add_node('run_tools', tool_node)
workflow.add_node('writer', writer_agent)
workflow.add_node('controller',controller_agent)
workflow.add_node('formatter',formatter)

workflow.add_edge(START,'researcher')
workflow.add_conditional_edges(
    'researcher',
    should_contunie,
    {
        'tool_call':'run_tools',
        'compile':'compile_research'
    },
)
workflow.add_edge('run_tools', 'researcher')
workflow.add_edge('compile_research','writer')
workflow.add_edge('writer','controller')

workflow.add_conditional_edges('controller',
                              decide_to_rewrite,
                              {
                                  "rewrite":"writer",
                                  'format':'formatter'
                              })

workflow.add_edge('formatter', END)
app = workflow.compile()

In [15]:
user_input = HumanMessage(content='what is the advantages and disadvantages of renewable energy sources?')
response = app.invoke({'messages': [user_input]})

--- RESEARCHER_agent WORKING ---
--- REASEARCHER FINISHED DRAFT ---
--- RESEARCHER_agent WORKING ---
--- REASEARCHER FINISHED DRAFT ---
--- compile_research_node WORKING ---
--- COMPAILE FINISHED DRAFT ---
--- writer_agent WORKING ---
--- WRITER FINISHED DRAFT ---
---CONTROLLER IS RUNNING---
---CONTROLLER FIND SOME ERROR:---
---CONTROLLER IS FINISHED---
--- writer_agent WORKING ---
--- WRITER FINISHED DRAFT ---
---CONTROLLER IS RUNNING---
---THERE IS NO ERROR.
---CONTROLLER IS FINISHED---
---FORMATTER NODE IS RUNNING ---
The student_number_homeworkname.docx saved succesfully!.


In [16]:
user_input = HumanMessage(content='10 interview questions and answers about machine learning.')
response = app.invoke({'messages': [user_input]})

--- RESEARCHER_agent WORKING ---
--- REASEARCHER FINISHED DRAFT ---
--- RESEARCHER_agent WORKING ---
--- REASEARCHER FINISHED DRAFT ---
--- compile_research_node WORKING ---
--- COMPAILE FINISHED DRAFT ---
--- writer_agent WORKING ---
--- WRITER FINISHED DRAFT ---
---CONTROLLER IS RUNNING---
---CONTROLLER FIND SOME ERROR:---
---CONTROLLER IS FINISHED---
--- writer_agent WORKING ---
--- WRITER FINISHED DRAFT ---
---CONTROLLER IS RUNNING---
---THERE IS NO ERROR.
---CONTROLLER IS FINISHED---
---FORMATTER NODE IS RUNNING ---
The student_number_homeworkname.docx saved succesfully!.
